In [2]:
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable
import pandas as pd

In [4]:
df = pd.DataFrame({
    'Store Location':['Delhi', 'Mumbai', 'Bangalore','Chennai', 'Kolkata', 'Hyderabad'],
    'Number of Employees':[51, 60, 43, 53, 43, 44],
    'Management Time/Week': [38, 45, 33, 43, 38, 35],
    'Number of Dresses Sold/Week': [169, 243, 173, 216, 155, 169],
    'Number of Accessories Sold/Week': [119, 167, 158, 138, 161, 157]})

In [13]:
df

,Store Location,Number of Employees,Management Time/Week,Number of Dresses Sold/Week,Number of Accessories Sold/Week
0,Delhi,51,38,169,119
1,Mumbai,60,45,243,167
2,Bangalore,43,33,173,158
3,Chennai,53,43,216,138
4,Kolkata,43,38,155,161
5,Hyderabad,44,35,169,157


In [19]:
model = LpProblem(name="small-problem", sense=LpMaximize)
Inputs = ["Number of Employees", 'Management Time/Week']
Outputs = ["Number of Dresses Sold/Week", "Number of Accessories Sold/Week"]

In [20]:
Inputs_Variables = []
Outputs_Variables = []
for input in Inputs:
    var = LpVariable(name=input, lowBound=0)
    Inputs_Variables.append(var)
for output in Outputs:
    var = LpVariable(name=output, lowBound=0)
    Outputs_Variables.append(var)

In [62]:
df['Inputs'] = df.apply(lambda x: lpSum([var * x[name] for (var, name) in zip(Inputs_Variables, Inputs)]), axis=1)
df['Outputs'] = df.apply(lambda x: lpSum([var * x[name] for (var, name) in zip(Outputs_Variables, Outputs)]), axis=1)
df['Outputs - Inputs'] = df.apply(lambda x: x['Outputs'] - x['Inputs'] <= 0, axis=1)
df['Inputs_St'] = df['Inputs'].apply(lambda x: x==1)

In [110]:
k = 0
def solve(k):
    pis = []
    model = LpProblem(name=f"small-problem_{k}", sense=LpMaximize)
    for i,sj in enumerate(df['Outputs - Inputs'].values):
        if i!=k:
            model += sj
    model+= df['Inputs_St'].values[k]
    model+= df['Outputs'].values[k]
    status = model.solve()
    for (name, constraint) in model.constraints.items():
        # print(f"{constraint}: {constraint.pi}")
        pis.append(constraint.pi)
    pis.insert(k, 0)
    return status, model.objective.value(), model, pis

In [126]:
pis

[-0.0, -0.0, 0.85459411, -0.0, 0.13648528, 0, 0.96855486]

In [127]:
models = []
datas =[]
for i in range(6):
    dx = []
    status, val, model, pis = solve(i)
    print('='*50)
    print(status, val, pis)
    dx = pis
    models.append(model)
    datas.append(dx)

1 0.82543740098 [0, 0.64348101, 0.07302957, -0.0, -0.0, -0.0, 0.8254374]
1 1.0300578029999998 [-0.0, 0, 1.4046243, -0.0, -0.0, -0.0, 1.0300578]
1 1.0782482327 [-0.0, 0.046232877, 0, -0.0, -0.0, 0.95719178, 1.0782482]
1 1.0062893088 [-0.0, 0.88888889, -0.0, 0, -0.0, -0.0, 1.0062893]
1 1.0189873379 [-0.0, -0.0, 1.0189873, -0.0, 0, -0.0, 1.0189873]
1 0.9685548570499999 [-0.0, -0.0, 0.85459411, -0.0, 0.13648528, 0, 0.96855486]


In [138]:
output = pd.DataFrame(datas,columns=df['Store Location'].to_list() +['DMU'])

In [140]:
output['Store Location'] = df['Store Location']

In [142]:
output = output[['Store Location', 'DMU']+df['Store Location'].to_list()]

In [143]:
output

,Store Location,DMU,Delhi,Mumbai,Bangalore,Chennai,Kolkata,Hyderabad
0,Delhi,0.825437,0.0,0.643481,0.073030,-0.0,-0.000000,-0.000000
1,Mumbai,1.030058,-0.0,0.000000,1.404624,-0.0,-0.000000,-0.000000
2,Bangalore,1.078248,-0.0,0.046233,0.000000,-0.0,-0.000000,0.957192
3,Chennai,1.006289,-0.0,0.888889,-0.000000,0.0,-0.000000,-0.000000
4,Kolkata,1.018987,-0.0,-0.000000,1.018987,-0.0,0.000000,-0.000000
5,Hyderabad,0.968555,-0.0,-0.000000,0.854594,-0.0,0.136485,0.000000
